# [1] Setting dataset

In [1]:
from Dataset import loadingFiles, cut_label, truncated_data, code_to_id, RETAIN_datasets
MIMICPath = 'Insert_into_your_data_path'

# Loading
treat_add_id = loadingFiles(MIMICPath, 'Final_treat_dx.pkl')
comparator_add_id = loadingFiles(MIMICPath, 'Final_comparator_dx.pkl')
treat_add_label = [seq for pid, seq in treat_add_id]

# Cutting label
treat = cut_label(treat_add_label)
comparator = [seq for pid, seq in comparator_add_id]

# Cut length <=2
print('\n## Cutting length one more..')
print('before: ', 'treat: ', len(treat), 'comparator: ', len(comparator))
treat = truncated_data(treat, cut_num=2)
comparator = truncated_data(comparator, cut_num=2)
print('after: ', 'treat: ', len(treat), 'comparator: ', len(comparator))
CKD_code_dict, max_visit_size = code_to_id(treat+comparator)


# MIMIC directory
import os
DATA_PATH = 'Insert_into_your_data_path'
DATA_PATH = os.path.join(DATA_PATH, 'MIMIC')
if not os.path.exists(DATA_PATH): os.makedirs(DATA_PATH)

# Reading MIMIC dataset
datasets = RETAIN_datasets(treat, comparator, CKD_code_dict, max_visit_size)

Loading at.. /Final_treat_dx.pkl
Loading at.. /Final_comparator_dx.pkl

## Cutting length one more..
before:  treat:  275 comparator:  1067
after:  treat:  275 comparator:  1067
code_size:  2872


# [2] Setting hyperparameter

In [2]:
batch_size = 10
time_size = max_visit_size
code_size = len(CKD_code_dict)
label_size = 2
embedding_size = 128
lr_init = 0.0001
decay_step = 2000
decay_rate = 0.9
training_step = 5000
printby = 100

hidden_size_alpha = 128
hidden_size_beta = 128

# [3] Setting model

In [3]:
import tensorflow as tf

def sequence_masking(data, visit_times):
    masking = tf.tile(tf.reshape(tf.sequence_mask(visit_times, data.shape[1]), shape=[-1,data.shape[1],1]), [1,1,data.shape[2]])
    return tf.where(masking, data, tf.zeros_like(data))

inputs = tf.placeholder(tf.float32, shape=[None, time_size, code_size])
labels = tf.placeholder(tf.float32, shape=[None, time_size, label_size])
visit_times = tf.placeholder(tf.float32, shape=[None])
global_step = tf.Variable(0, trainable=False, dtype=tf.int32)
lr = tf.train.exponential_decay(lr_init, global_step, decay_step, decay_rate, staircase=True)

##embedding
W_emb = tf.Variable(tf.random_normal(shape=[code_size, embedding_size]), name='W_emb')
b_emb = tf.Variable(tf.random_normal(shape=[embedding_size]), name='b_emb')
reshaped_inputs = tf.reshape(inputs, shape=[-1, code_size])
embedded = tf.matmul(reshaped_inputs, W_emb) + b_emb
embedded = tf.reshape(embedded, shape=[-1, time_size, embedding_size])
v_ = tf.nn.tanh(embedded)

##alpha
W_alpha = tf.Variable(tf.random_normal(shape=[hidden_size_alpha, 1], name='W_alpha'))
b_alpha = tf.Variable(tf.random_normal(shape=[], name='b_alpha'))
with tf.variable_scope('alpha'):
    cells_alpha = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(hidden_size_alpha)])
    outputs_alpha, states_alpha = tf.nn.dynamic_rnn(cells_alpha, v_, visit_times, dtype=tf.float32)
    reshaped_outputs_alpha = tf.reshape(outputs_alpha, shape=[-1, hidden_size_alpha])
    matmuled_alpha = tf.matmul(reshaped_outputs_alpha, W_alpha) + tf.expand_dims(b_alpha, 0)
    reshaped_matmuled_alpha = tf.reshape(matmuled_alpha, shape=[-1, time_size, 1])
    logits_alpha = tf.nn.softmax(reshaped_matmuled_alpha, 1)
    
##Beta
W_beta = tf.Variable(tf.random_normal(shape=[hidden_size_beta, code_size]))
b_beta = tf.Variable(tf.random_normal(shape=[code_size]))
with tf.variable_scope('beta'):
    cells_beta = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(hidden_size_beta)])
    outputs_beta, states_beta = tf.nn.dynamic_rnn(cells_beta, v_, visit_times, dtype=tf.float32)
    reshaped_outputs_beta = tf.reshape(outputs_beta, shape=[-1, hidden_size_beta])
    matmuled_ouputs_beta = tf.matmul(reshaped_outputs_beta, W_beta) + tf.expand_dims(b_beta, 0)
    reshaped_matmuled_beta = tf.reshape(matmuled_ouputs_beta, shape=[-1, time_size, code_size])
    logits_beta = tf.nn.tanh(reshaped_matmuled_beta)
    
##unifiy
W_s = tf.Variable(tf.random_normal(shape=[code_size, label_size]))
b_s = tf.Variable(tf.random_normal(shape=[label_size]))
unified_context = logits_alpha*logits_beta*inputs
context_vec = []
for b in range(batch_size):
    pid_seq = unified_context[b]
    for t in range(time_size):
        context_vec.append(tf.reduce_sum(pid_seq[:t+1, :], axis=0))
reshaped_context_vec = tf.reshape(context_vec, shape=[-1, code_size])
mlp_context_vec = tf.matmul(reshaped_context_vec, W_s) + b_s
logits = tf.nn.softmax(tf.reshape(mlp_context_vec, shape=[-1, time_size, label_size]))
masked_logits = sequence_masking(logits, visit_times)
masked_labels = sequence_masking(labels, visit_times)

##Loss function
loss_per_times = tf.reduce_sum(masked_labels*tf.log(masked_logits+1e-10)+(1-masked_labels)*tf.log(1-masked_logits+1e-10), axis=-1)
loss_per_pid = tf.reduce_sum(loss_per_times, axis=-1)*(1/visit_times)
last_loss = -tf.reduce_sum(loss_per_pid)*(1/batch_size)
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(last_loss, global_step=global_step)

# [4] Training & Testing

In [4]:
from tqdm import trange
with tf.Session() as sess:
    tf.train.start_queue_runners(sess=sess)
    tf.global_variables_initializer().run(session=sess)

    for step in trange(training_step):
        batch_train_inputs, batch_train_labels, batch_train_visit_times = datasets.train.next_batch(batch_size)
        batch_val_inputs, batch_val_labels, batch_val_visit_times = datasets.validation.next_batch(batch_size)
        batch_te_inputs, batch_te_labels, batch_te_visit_times = datasets.test.next_batch(batch_size)
        train_feed_dict = {inputs: batch_train_inputs, 
                           labels: batch_train_labels, 
                           visit_times: batch_train_visit_times}
        validation_feed_dict = {inputs: batch_val_inputs, 
                                labels: batch_val_labels, 
                                visit_times: batch_val_visit_times}
        test_feed_dict = {inputs: batch_te_inputs, 
                          labels: batch_te_labels, 
                          visit_times: batch_te_visit_times}
        _, g_step, train_loss, train_lr_ = sess.run([optimizer, global_step, last_loss, lr], feed_dict=train_feed_dict)
        val_loss, val_lr_ = sess.run([last_loss, lr], feed_dict=validation_feed_dict)
        
        if step % printby == 0:
            print('step: {} \tg_step: {} \tloss: {:.8f}||{:.8f} \tlr: {:.8f}||{:.8f}'.format(step, g_step, train_loss, val_loss, train_lr_, val_lr_))
        
        if step == training_step-1:
            te_loss = sess.run(last_loss, feed_dict=test_feed_dict)
            print('All Done! test_loss is {}'.format(te_loss))

  0%|          | 2/5000 [00:03<3:31:32,  2.54s/it]

step: 0 	g_step: 1 	loss: 1.77771223||1.76532292 	lr: 0.00010000||0.00010000


  2%|▏         | 101/5000 [00:25<17:01,  4.80it/s]

step: 100 	g_step: 101 	loss: 1.36491549||1.72819364 	lr: 0.00010000||0.00010000


  4%|▍         | 201/5000 [00:44<15:15,  5.24it/s]

...

step: 4800 	g_step: 4801 	loss: 0.01429659||0.76042825 	lr: 0.00008100||0.00008100


 98%|█████████▊| 4902/5000 [15:36<00:15,  6.13it/s]

step: 4900 	g_step: 4901 	loss: 0.04870249||3.22787261 	lr: 0.00008100||0.00008100


100%|██████████| 5000/5000 [15:52<00:00,  2.99it/s]

All Done! test_loss is 2.0485293865203857


# [5] Training Reversed dataset

In [6]:
from Dataset import reverse_seq

rvrsed_treat = reverse_seq(treat)
rvrsed_comparator = reverse_seq(comparator)

rvrsed_datasets = RETAIN_datasets(rvrsed_treat, rvrsed_comparator, CKD_code_dict, max_visit_size)

In [7]:
from tqdm import trange
with tf.Session() as sess:
    tf.train.start_queue_runners(sess=sess)
    tf.global_variables_initializer().run(session=sess)

    for step in trange(training_step):
        batch_train_inputs, batch_train_labels, batch_train_visit_times = rvrsed_datasets.train.next_batch(batch_size)
        batch_val_inputs, batch_val_labels, batch_val_visit_times = rvrsed_datasets.validation.next_batch(batch_size)
        batch_te_inputs, batch_te_labels, batch_te_visit_times = rvrsed_datasets.test.next_batch(batch_size)
        train_feed_dict = {inputs: batch_train_inputs, 
                           labels: batch_train_labels, 
                           visit_times: batch_train_visit_times}
        validation_feed_dict = {inputs: batch_val_inputs, 
                                labels: batch_val_labels, 
                                visit_times: batch_val_visit_times}
        test_feed_dict = {inputs: batch_te_inputs, 
                          labels: batch_te_labels, 
                          visit_times: batch_te_visit_times}
        _, g_step, train_loss, train_lr_ = sess.run([optimizer, global_step, last_loss, lr], feed_dict=train_feed_dict)
        val_loss, val_lr_ = sess.run([last_loss, lr], feed_dict=validation_feed_dict)
        
        if step % printby == 0:
            print('step: {} \tg_step: {} \tloss: {:.8f}||{:.8f} \tlr: {:.8f}||{:.8f}'.format(step, g_step, train_loss, val_loss, train_lr_, val_lr_))
        
        if step == training_step-1:
            te_loss = sess.run(last_loss, feed_dict=test_feed_dict)
            print('All Done! test_loss is {}'.format(te_loss))

  0%|          | 2/5000 [00:03<3:00:17,  2.16s/it]

step: 0 	g_step: 1 	loss: 1.89690912||1.69452178 	lr: 0.00010000||0.00010000


  2%|▏         | 102/5000 [00:20<14:16,  5.72it/s]

step: 100 	g_step: 101 	loss: 1.48023438||1.74885869 	lr: 0.00010000||0.00010000


  4%|▍         | 202/5000 [00:40<15:56,  5.01it/s]

...

step: 4800 	g_step: 4801 	loss: 0.01473635||4.33526802 	lr: 0.00008100||0.00008100


 98%|█████████▊| 4902/5000 [14:13<00:16,  5.88it/s]

step: 4900 	g_step: 4901 	loss: 0.10136785||2.70155931 	lr: 0.00008100||0.00008100


100%|██████████| 5000/5000 [14:29<00:00,  2.95it/s]

All Done! test_loss is 2.3594472408294678
